# Assignment #5 - Artificial Neural Networks
---
## Name: 李勝維
## Student ID: 0711239
---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pytorch_lightning import seed_everything
import warnings;warnings.filterwarnings("ignore")
%matplotlib inline
seed_everything(0)
plt.rcParams['figure.dpi'] = 500

# Data Input & Data Visualization

Use package 'json' to read and 'pandas' to save data  

In [ ]:
import json
# Data input
with open('2021-nycu-ml-hw5\\train.json', 'r') as file:
    raw_data = json.load(file)
raw_data = [(x['id'], x['cuisine'], x['ingredients']) for x in raw_data]
ids, cuisines, ingredientss = zip(*raw_data)
df = pd.DataFrame({
    'id':ids,
    'cuisine':cuisines,
    'ingredients':ingredientss,
})

# Data Preprocessing

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Shuffle data
df = df.sample(frac=1, random_state=0)

# Encode Y
y_encoder = LabelEncoder()
df['cuisine'] = y_encoder.fit_transform(df['cuisine'])
y = df['cuisine'].values.astype(np.longlong)